In [1]:
# Pendientes
# Formatear nombres de las columnas para eliminar espacios
# Hacer iteraciones para aplicar las funciones sobre múltiples datasets
# Simplificar la función de lectura y formateo de columnas

In [2]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np

In [3]:
# Definir la ruta de los datos

# Sitio norte
fp_north_1 = '../../data/raw/soil-sensors/z6-25818/z6-25818(z6-25818)-Configuration 3-1756238685.7139447.csv'
fp_north_2 = '../../data/raw/soil-sensors/z6-25818/z6-25818(z6-25818)-Configuration 4-1756238685.7139447.csv'

# Sitio sur
fp_south_1 = '../../data/raw\soil-sensors/z6-26092/z6-26092(z6-26092)-Configuration 3-1756238687.030961.csv'
fp_south_2 = '../../data/raw\soil-sensors/z6-26092/z6-26092(z6-26092)-Configuration 4-1756238687.030961.csv'

In [4]:
# Asignar la profundidad de cada instrumento a los puertos del datalogger

# Sitio norte
port_mapping_north = {
    "Port1": "TEROS12 -48cm",
    "Port2": "TEROS12 -30cm",
    "Port3": "TEROS12 -15cm",
    "Port4": "TEROS21 -35cm",
    "Port5": "TEROS21 -25cm",
}

# Sitio sur
port_mapping_south = {
    "Port1": "TEROS12 -65cm",
    "Port2": "TEROS12 -45cm",
    "Port3": "TEROS12 -18cm",
    "Port4": "TEROS21 -55cm",
    "Port5": "TEROS21 -31cm",
}

In [5]:
# Funcion para leer los datos, renombrar los encabezados de las tablas
# y formatear los datos al tipo correcto (numerico, fecha)

def read_and_convert_data(filepath, port_mapping):

    # Leer el archivo CSV
    df = pd.read_csv(filepath, header=None, low_memory=False)

    # Crear una lista vacia para las nuevas columnas
    new_columns = []

    # Iterar usando el indice de cada columna del df
    for i in range(df.shape[1]):
        # Extraer el nombre del puerto (ej. 'Port1') de la primera fila
        port = str(df.iloc[0, i])
        # Extraer la medida y unidad (ej. 'm3/m3 Water Content') de la tercera fila
        measurement = str(df.iloc[2, i])

        # Usar el mapeo si el puerto existe; de lo contrario, usar el nombre original del puerto
        descriptive_name = port_mapping.get(port, port)

        # La primera columna es 'Timestamps', por lo que se deja ese nombre
        if i == 0:
            final_name = measurement
        else:
            # Crear el nombre final de la columna combinando el nombre descriptivo y la medida
            final_name = f"{descriptive_name}{measurement}"
            
        new_columns.append(final_name)

    # Asignar la nueva lista de nombres de columna al df
    df.columns = new_columns

    # Eliminar las 3 primeras filas del df
    df = df.iloc[3:].reset_index(drop=True)

    # Convertir la columna de timestamps a datetime
    df['Timestamps'] = pd.to_datetime(df['Timestamps'])

    # Convertir todas las demás columnas a tipos numericos.
    for col in df.columns:
        if col != 'Timestamps':
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Establecer la columna 'Timestamps' como indice
    df = df.set_index("Timestamps")
    
    return df

In [6]:
# Ejecutar la funcion de lectura y conversión de tipos de datos

# Sitio norte
north_soil_data_1 = read_and_convert_data(fp_north_1, port_mapping_north)
north_soil_data_2 = read_and_convert_data(fp_north_2, port_mapping_north)

# Sitio sur
south_soil_data_1 = read_and_convert_data(fp_south_1, port_mapping_south)
south_soil_data_2 = read_and_convert_data(fp_south_2, port_mapping_south)

In [7]:
# Unir los df de cada sitio en uno solo, y descartar las columnas no compartidas
north_soil_data = pd.concat([north_soil_data_1, north_soil_data_2], join="inner")
south_soil_data = pd.concat([south_soil_data_1, south_soil_data_2], join="inner")

In [8]:
# Identificar registros duplicados
duplicated_north = north_soil_data[north_soil_data.index.duplicated(keep=False)]
duplicated_south = south_soil_data[south_soil_data.index.duplicated(keep=False)]

# print(duplicated_north)
# print(duplicated_south)

In [9]:
# Función para rellenar con NaNs los saltos temporales del df

def reindex_dataframes(date_filtered_df, interval):

    # Generar un índice a intervalo regular del periodo completo
    full_index = pd.date_range(
        start=date_filtered_df.index.min(),
        end=date_filtered_df.index.max(),
        freq=interval,
        name=date_filtered_df.index.name
    )

    # Reindexar el date_filtered_df usando el full index
    reindexed_df = date_filtered_df.reindex(full_index)

    return reindexed_df

In [10]:
# Filtrar dfs por periodo de funcionamiento de cada intervalo de medicion

# Sitio norte
north_15min = north_soil_data.loc[north_soil_data.index.min():'2024-10-17 14:30:00']
north_10min = north_soil_data.loc['2024-10-17 14:30:01':north_soil_data.index.max()]

# Sitio sur
south_15min = south_soil_data.loc[south_soil_data.index.min():'2024-10-17 23:00:00']
south_10min = south_soil_data.loc['2024-10-17 23:00:01':south_soil_data.index.max()]

In [11]:
# Ejecutar función de reindexación

# Sitio norte
north_15min_reindexed = reindex_dataframes(north_15min, '15min')
north_10min_reindexed = reindex_dataframes(north_10min, '10min') 

# Sitio sur
south_15min_reindexed = reindex_dataframes(south_15min, '15min')
south_10min_reindexed = reindex_dataframes(south_10min, '10min')    # 2024-10-17 23:10 no se crea

In [12]:
# Concatenar los dataframes para re-unificarlos
north_soil_data = pd.concat([north_15min_reindexed, north_10min_reindexed])
south_soil_data = pd.concat([south_15min_reindexed, south_10min_reindexed])

In [13]:
# Exportar datos procesados como csv

north_soil_data.to_csv('../../data/processed/cleaned/soil_SDH1_cleaned.csv')
south_soil_data.to_csv('../../data/processed/cleaned/soil_SDH2_cleaned.csv')